In [ ]:
!pip install selenium webdriver-manager beautifulsoup4


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 14.3 MB/s eta 0:00:00


In [ ]:
import requests
import csv
import pandas as pd
import os
import time
import random
from bs4 import BeautifulSoup
from google.colab import files

# Definir la URL base de búsqueda en eBay con paginación
search_url = "https://www.ebay.com/sch/i.html?_nkw={}&_sop=12&_pgn={}"

# Lista de términos de búsqueda ampliada para obtener más productos
search_terms = ["electronics", "home decor", "clothing", "toys", "beauty products", "kitchen appliances", "sports equipment", "books", "automotive accessories", "gaming", "smartphones", "watches", "furniture", "office supplies", "pet supplies"]

# Número de páginas a recorrer por cada término de búsqueda
num_pages = 20  # Aumentado para obtener más observaciones

# Lista para almacenar los productos
productos = []

# Headers para evitar bloqueos por parte de eBay
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
}

for term in search_terms:
    for page in range(1, num_pages + 1):
        url = search_url.format(term.replace(" ", "+"), page)
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")

            for producto in soup.find_all("li", class_="s-item"):
                try:
                    nombre_tag = producto.find("h3", class_="s-item__title")
                    if not nombre_tag or "Shop on eBay" in nombre_tag.text:
                        continue  # Omitir elementos sin nombre válido
                    nombre = nombre_tag.text.strip()

                    precio_tag = producto.find("span", class_="s-item__price")
                    precio = precio_tag.text.strip() if precio_tag else "N/A"

                    categoria = term.title()
                    disponibilidad = "Disponible"

                    ventas_tag = producto.find("span", class_="s-item__hotness")
                    ventas = ventas_tag.text.strip() if ventas_tag else "N/A"

                    num_resenas_tag = producto.find("span", class_="s-item__reviews-count")
                    num_resenas = num_resenas_tag.text.strip() if num_resenas_tag else "N/A"

                    calificacion_tag = producto.find("div", class_="x-star-rating")
                    calificacion = calificacion_tag.text.strip() if calificacion_tag else "N/A"

                    # Extraer unidades vendidas
                    unidades_vendidas_tag = producto.find("span", class_="s-item__quantitySold")
                    unidades_vendidas = unidades_vendidas_tag.text.strip() if unidades_vendidas_tag else "N/A"

                    # Extraer tiempo de entrega estimado
                    tiempo_entrega_tag = producto.find("span", class_="s-item__logisticsCost")
                    tiempo_entrega = tiempo_entrega_tag.text.strip() if tiempo_entrega_tag else "N/A"

                    # Extraer número de watchers (personas interesadas en el producto)
                    watchers_tag = producto.find("span", class_="s-item__watchCount")
                    watchers = watchers_tag.text.strip() if watchers_tag else "N/A"

                    productos.append([nombre, categoria, precio, disponibilidad, ventas, num_resenas, calificacion, unidades_vendidas, tiempo_entrega, watchers])
                except AttributeError:
                    continue
        else:
            print(f"Error al acceder a {url}, código de estado: {response.status_code}")

        # Espera aleatoria para evitar bloqueos
        time.sleep(random.uniform(1, 3))

# Guardar en CSV
csv_filename = "inventario_ebay_extendido.csv"
df = pd.DataFrame(productos, columns=["Nombre", "Categoría", "Precio", "Disponibilidad", "Ventas Recientes", "Número de Reseñas", "Calificación Promedio", "Unidades Vendidas", "Tiempo de Entrega", "Watchers"])
df.to_csv(csv_filename, index=False, encoding='utf-8')

# Verificar si hay datos antes de intentar descargar
if not df.empty:
    print(f"Datos guardados en {csv_filename} con {len(df)} registros.")
    files.download(csv_filename)
else:
    print("No se encontraron productos. eBay podría estar bloqueando el scraping.")

No se encontraron productos. eBay podría estar bloqueando el scraping.
